In [1]:
from vk_api import VkApi
import vk_api
import vk
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
# from nltk.tokenize.casual import TweetTokenizer
# from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
import pymorphy2
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from keras.models import Sequential
from keras.layers import Dense, Dropout
from nltk.corpus import stopwords

# Прогноз количества комментариев постов 

In [2]:
# id групп вк

id_avia = '-39283725'
id_moscow = '-137324015'
id_nazem = '-212086355'
mas_id = [id_avia, id_moscow, id_nazem]



Парсинг данных из групп вк

In [3]:

with open('C:/Users/Admin/Desktop/token.txt') as f:
     token = f.readlines()[0]


def main(offset: int, token: str, group_id: str):
    vk = vk_api.VkApi(token=token)  # авторизация через токен (желательно)
    api = vk.get_api()
    posts = api.wall.get(owner_id=group_id, offset=offset, count=100)['items']
    posts_strings = [post['text'] for post in posts]
    num_like = []
    comments_strings = []
    for post in posts:
        comments = api.wall.getComments(
            owner_id=group_id, post_id=post['id'], count=100)['items']
        comments_strings.append([comment['text'] for comment in comments])
        itemID = post['id']
        isLiked = api.likes.getList(
            type='post',
            owner_id=group_id,
            item_id=itemID
        )
        num_like.append(isLiked['count'])
    return posts_strings, comments_strings, num_like


combo_list_posts = []
combolist_comments = []
combolist_like = []

for group_id in mas_id:
    for i in range(0, 301, 100):
        try:
            rzd_posts, comments_strings_rzd, rzd_like_count = main(
                offset=i, token=token, group_id=group_id)
            combo_list_posts.extend(rzd_posts)
            combolist_comments.extend(comments_strings_rzd)
            combolist_like.extend(rzd_like_count)
            print(group_id, i)
        except:
            print('Постов больше нет на смещении: ', i)

       


-39283725 0
-39283725 100
-39283725 200
-39283725 300
-137324015 0
-137324015 100
-137324015 200
-137324015 300
-212086355 0
-212086355 100
-212086355 200
-212086355 300


In [4]:
combolist_comments_count = [len(comments) for comments in combolist_comments]


In [5]:
df = pd.DataFrame({'Посты': combo_list_posts,
                  'Комментарии': combolist_comments_count})
df


,Посты,Комментарии
0,"Казанский вертолетный завод (входит в ""Вертоле...",1
1,Авиакомпания Smartwings стала полностью чешско...,1
2,В новом весенне-летнем сезоне география полето...,0
3,ЦАГИ участвует в разработке технологии энергет...,0
4,"Компания ""ТКП — Центральная Азия"" (ТКП-ЦА), пл...",0
...,...,...
1195,Как с начала года менялся наземный транспорт в...,4
1196,⚡️ Полностью перевели Дом для электробусов «Ми...,12
1197,Больше 183 млн посмотрели расписание наземного...,6
1198,🚌 Объявляем конкурс в честь 2-летия сервиса пе...,7


In [6]:
df = df.drop_duplicates()
df.isnull().sum()



Посты          0
Комментарии    0
dtype: int64

In [7]:
df['Комментарии'].mean()


5.648323301805675

In [8]:
df = df.astype({'Посты': np.str})


C:\Users\Admin\AppData\Local\Temp\ipykernel_23324\2142041351.py:1: DeprecationWarning: `np.str` is a deprecated alias for the builtin `str`. To silence this warning, use `str` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.str_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  df = df.astype({'Посты': np.str})


Токенизация

In [9]:

nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

stop_words = set(stopwords.words('russian'))
lemmatizer = WordNetLemmatizer()
morph = pymorphy2.MorphAnalyzer()


def preprocess_text(text):
    # Токенизация
    tokens = word_tokenize(text)

    # Приведение к нижнему регистру
    tokens = [token.lower() for token in tokens]

    # Лемматизация
    tokens = [lemmatizer.lemmatize(token) for token in tokens]

    # Приведение к начальной форме
    tokens = [morph.parse(token)[0].normal_form for token in tokens]

    # Удаление стоп-слов
    tokens = [token for token in tokens if token not in stop_words]

    # Объединение токенов в строку
    clean_text = ' '.join(tokens)

    return clean_text


df['clean_text'] = df['Посты'].apply(preprocess_text)


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [10]:
df

,Посты,Комментарии,clean_text
0,"Казанский вертолетный завод (входит в ""Вертоле...",1,казанский вертолётный завод ( входить `` верто...
1,Авиакомпания Smartwings стала полностью чешско...,1,авиакомпания smartwings стать полностью чешски...
2,В новом весенне-летнем сезоне география полето...,0,новый весенне-летний сезон география полёт `` ...
3,ЦАГИ участвует в разработке технологии энергет...,0,цаги участвовать разработка технология энергет...
4,"Компания ""ТКП — Центральная Азия"" (ТКП-ЦА), пл...",0,компания `` ткп — центральный азия '' ( ткп-ца...
...,...,...,...
1195,Как с начала года менялся наземный транспорт в...,4,начало год меняться наземный транспорт новый м...
1196,⚡️ Полностью перевели Дом для электробусов «Ми...,12,⚡️ полностью перевести дом электробус « митино...
1197,Больше 183 млн посмотрели расписание наземного...,6,большой 183 млн посмотреть расписание наземный...
1198,🚌 Объявляем конкурс в честь 2-летия сервиса пе...,7,🚌 объявлять конкурс честь 2-летие сервис перев...


In [11]:

text_data = df['clean_text']
labels = df['Комментарии']

# Разделение данных на тренировочный и тестовый наборы
X_train, X_test, y_train, y_test = train_test_split(
    text_data, labels, test_size=0.2, random_state=42)

# Векторизация текстовых данных
vectorizer = TfidfVectorizer()
X_train = vectorizer.fit_transform(X_train).toarray() 
X_test = vectorizer.transform(X_test).toarray()

# Создание модели нейронной сети
model = Sequential()
model.add(Dense(100, activation='relu', input_dim=X_train.shape[1]))
model.add(Dropout(0.2))
model.add(Dense(64, activation='relu'))
model.add(Dense(1, activation='linear'))

# Компиляция модели
model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mae'])

# Обучение модели
model.fit(X_train, y_train, epochs=10, batch_size=64)



Epoch 1/10
15/15 [==============================] - 1s 13ms/step - loss: 90.5238 - mae: 5.6016
Epoch 2/10
15/15 [==============================] - 0s 14ms/step - loss: 81.7267 - mae: 5.0074
Epoch 3/10
15/15 [==============================] - 0s 13ms/step - loss: 65.4811 - mae: 4.4680
Epoch 4/10
15/15 [==============================] - 0s 16ms/step - loss: 50.4297 - mae: 4.2025
Epoch 5/10
15/15 [==============================] - 0s 15ms/step - loss: 41.2721 - mae: 3.8314
Epoch 6/10
15/15 [==============================] - 0s 14ms/step - loss: 34.1847 - mae: 3.1345
Epoch 7/10
15/15 [==============================] - 0s 14ms/step - loss: 28.0557 - mae: 2.6644
Epoch 8/10
15/15 [==============================] - 0s 14ms/step - loss: 21.7284 - mae: 2.1619
Epoch 9/10
15/15 [==============================] - 0s 14ms/step - loss: 18.1854 - mae: 1.8912
Epoch 10/10
15/15 [==============================] - 0s 14ms/step - loss: 14.0082 - mae: 1.5886


In [12]:
loss, mae = model.evaluate(X_test, y_test)
print(f'Test Loss: {loss}, Test MAE: {mae}')


8/8 [==============================] - 0s 4ms/step - loss: 34.8395 - mae: 3.1047
Test Loss: 34.839534759521484, Test MAE: 3.1047074794769287
